In [1]:
%pwd

'/home/techie/Desktop/general/end-to-end-kidney-tumor/research'

In [2]:
import os 

os.chdir('../')
%pwd

'/home/techie/Desktop/general/end-to-end-kidney-tumor'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class CallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    model_trained_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from kidney.constants import PARAMS_PATH, CONFIG_PATH
from kidney.utils.common import read_yaml, create_directories
import os

In [5]:
class ConfigurationManager:
    def __init__(self, 
                 config=CONFIG_PATH, 
                 params=PARAMS_PATH) -> None:
        self.config = read_yaml(config)
        self.params = read_yaml(params)
        
        
    def get_call_back_config(self)-> CallBackConfig:
        config = self.config.call_backs
        create_directories([config.root_dir])
        
        call_back_config = CallBackConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )
        return call_back_config
        
    
    def get_model_training_config(self)-> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params
        base_model = self.config.base_model
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        
        create_directories([config.root_dir])
        
        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            model_trained_path=config.model_trained_path,
            updated_base_model_path=base_model.updated_base_model_path,
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return model_training_config

In [6]:
import time
import tensorflow as tf


2023-11-15 08:43:44.747819: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 08:43:44.900135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 08:43:44.902388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 08:43:47.536503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
class CallBacks:
    def __init__(self, config: CallBackConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
class ModelTraining:
    def __init__(self, config:ModelTrainingConfig) -> None:
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
    def train_val_gen(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_data_generator = ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_gen = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_data_generator = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_data_generator = ImageDataGenerator(
                **datagenerator_kwargs
            )
            
        self.train_gen = train_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self, callback_list: list):
        self.model.fit(
            self.train_gen,
            epochs=self.config.params_epochs,
            validation_data=self.valid_gen,
            callbacks=callback_list
        )
        
        self.save_model(
            path=self.config.model_trained_path,
            model=self.model
        )
        
        

In [16]:
try:
    config = ConfigurationManager()
    config = ConfigurationManager()
    call_backs_config = config.get_call_back_config()
    call_backs = CallBacks(config=call_backs_config)
    callback_list = call_backs.get_tb_ckpt_callbacks()

    training_config = config.get_model_training_config()
    training = ModelTraining(config=training_config)
    training.get_base_model()
    training.train_val_gen()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e

[2023-11-15 08:47:37,469: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-15 08:47:37,477: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-15 08:47:37,488: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-15 08:47:37,495: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-15 08:47:37,498: INFO: common: created directory at: artifacts/call_backs]
[2023-11-15 08:47:37,501: INFO: common: created directory at: artifacts/model_training]


Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


2023-11-15 08:47:41.580185: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2023-11-15 08:47:43.203358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-11-15 08:47:47.470382: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2023-11-15 08:47:47.646889: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-15 08:47:49.142065: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-15 08:47:49.151787: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% 

12/12 [==============================] - ETA: 0s - loss: 13.4216 - accuracy: 0.5215 

2023-11-15 08:53:33.519488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


12/12 [==============================] - 439s 37s/step - loss: 13.4216 - accuracy: 0.5215 - val_loss: 18.6935 - val_accuracy: 0.4839
